In [1]:
# Step 1: Importing libraries and loading the data
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import MinMaxScaler

In [2]:
def normalized_func(data):
  # Convert 'DateTime' to datetime format
  data['DateTime'] = pd.to_datetime(data['DateTime'])
  
  # Selecting numerical columns for normalization
  normalize_features = ['WindSpeed(m/s)', 'Pressure(hpa)', 'Temperature(°C)', 'Humidity(%)', 'Sunlight(Lux)']
  data_numerical = data[normalize_features]

  scaler = MinMaxScaler(feature_range=(0, 1))
  data_normalized = scaler.fit_transform(data_numerical)

  # Creating a DataFrame with normalized data
  data_normalized = pd.DataFrame(data_normalized, columns=normalize_features)

  # Concatenate normalized data with other non-numerical columns if needed
  data_normalized = pd.concat([data[['LocationCode', 'DateTime', 'Power(mW)','Hour',
       'Minute', 'Month', 'Hour_sin', 'Hour_cos', 'Minute_sin', 'Minute_cos',
       'Month_sin', 'Month_cos', 'DayOfYear', 'DayOfYear_sin',
       'DayOfYear_cos']], data_normalized], axis=1)

  return data_normalized
  # print(data_normalized.head())

In [14]:
def resample_filter_func(data:pd.DataFrame):
  # Set DateTime as the index for resampling
  data.set_index('DateTime', inplace=True)

  # Resample data in 10-minute intervals using mean
  # 'LocationCode' is a non-numeric column, so we need to handle it separately if we have multiple locations.
  data_resampled = data.resample('1min').mean()

  data_resampled = data_resampled.between_time("07:00", "16:59")

  # Reset index if you want DateTime as a regular column
  data_resampled.reset_index(inplace=True)

  data_resampled.dropna(inplace=True)

  data_resampled['Date'] = data_resampled['DateTime'].dt.date
  
  data_resampled = data_resampled.groupby('Date').filter(lambda x: len(x)==600)
  
  data_resampled = data_resampled.drop(columns=['Date'])
  
  return data_resampled

  # print(data_resampled.head())

In [15]:
def process_pipeline(data_path:str, save_path:str):
  # Load the dataset (replace 'your_data.csv' with the actual file path)
  data = pd.read_csv(data_path)
  data_normalized = normalized_func(data)
  data_resampled = resample_filter_func(data_normalized)
  # Save the resampled data to a new CSV file
  data_resampled.to_csv(save_path, index=False)

In [16]:
# def process_all(dataset_num:int):
#   for step in range(1,dataset_num+1):
#     data_path = f"/home/sebastian/Desktop/AICUP-2024-Power_Prediciton/dataset/36_TrainingData_raw/L{step}_Train.csv"
#     save_path = f"/home/sebastian/Desktop/AICUP-2024-Power_Prediciton/dataset/36_TrainingData_process/L{step}_Train_resampled.csv"
#     process_pipeline(data_path,save_path)

In [17]:
def process_all(dataset_num:int):
  for step in range(1,dataset_num+1):
    data_path = f"/home/sebastian/Desktop/AICUP-2024-Power_Prediciton/dataset/36_TrainingData_interpolation/new_L{step}_Train.csv"
    save_path = f"/home/sebastian/Desktop/AICUP-2024-Power_Prediciton/dataset/36_TrainingData_interpolation_5min/L{step}_Train_resampled.csv"
    process_pipeline(data_path,save_path)

In [18]:
process_all(17)

In [20]:
data_path = f"/home/sebastian/Desktop/AICUP-2024-Power_Prediciton/dataset/36_TrainingData_interpolation/new_L1_Train.csv"
# data_path = f"/home/sebastian/Desktop/AICUP-2024-Power_Prediciton/dataset/36_TrainingData_interpolation_5min/L1_Train_resampled.csv"
data = pd.read_csv(data_path)
data.describe()

,LocationCode,WindSpeed(m/s),Pressure(hpa),Temperature(°C),Humidity(%),Sunlight(Lux),Power(mW),Hour,Minute,Month,Hour_sin,Hour_cos,Minute_sin,Minute_cos,Month_sin,Month_cos,DayOfYear,DayOfYear_sin,DayOfYear_cos
count,82708.0,82708.000000,82708.000000,82708.000000,82708.000000,82708.000000,82708.000000,82708.000000,82708.000000,82708.000000,82708.000000,82708.000000,82708.000000,8.270800e+04,82708.000000,82708.000000,82708.000000,82708.000000,82708.000000
mean,1.0,0.366623,1009.379480,31.696232,67.400872,32637.143473,363.005690,11.438059,29.478261,3.713897,0.107976,-0.722637,-0.000770,1.089048e-03,0.477679,-0.169699,97.986023,0.504215,-0.049460
std,0.0,0.854776,63.845463,10.143307,25.952298,30411.180597,520.429154,2.925557,17.338959,2.051924,0.639223,0.239869,0.706536,7.076849e-01,0.450497,0.734906,62.348830,0.364328,0.781406
min,1.0,0.000000,958.150000,10.500000,18.900000,148.330000,0.010000,7.000000,0.000000,1.000000,-0.965926,-1.000000,-1.000000,-1.000000e+00,-0.500000,-1.000000,1.000000,-0.393590,-0.999963
25%,1.0,0.000000,1003.760000,23.000000,43.400000,9296.460000,18.000000,9.000000,14.000000,2.000000,-0.500000,-0.965926,-0.669131,-6.691306e-01,0.500000,-0.866025,40.000000,0.221922,-0.865307
50%,1.0,0.000000,1007.980000,31.500000,62.800000,21352.500000,95.685000,11.000000,29.000000,4.000000,0.258819,-0.707107,0.000000,2.832769e-16,0.500000,-0.500000,93.000000,0.566702,-0.030120
75%,1.0,0.170000,1013.650000,39.700000,100.000000,47926.247500,496.067500,14.000000,44.000000,5.000000,0.707107,-0.500000,0.669131,7.431448e-01,0.866025,0.500000,152.000000,0.821477,0.772157
max,1.0,8.880000,9009.000000,61.800000,300.000000,125926.640625,2598.450000,17.000000,59.000000,7.000000,0.965926,-0.258819,1.000000,1.000000e+00,1.000000,0.866025,206.000000,0.999991,0.999852
